# 공동 작업

## 기준 브랜치 반영 작업

- 기준 브랜치를 여러 사람이 공동으로 사용하는 경우, 
- 개발자 A가 기준 브랜치에 작업 브랜치를 반영하는 시점에 이미 다른 사람인 B가 기준 브랜치에 먼저 반영한 내용이 있을 수 있다. 
- 이 때는 작업 내용에 따라 A가 기준 브랜치에 작업 브랜치를 반영하는 작업이 충돌할 수 있다.
- 기준 브랜치에 작업 브랜치를 반영하기 전에 먼저 기준 브랜치를 작업 브랜치에 반영하는 작업을 하면  

### 경우 1: 충돌이 발생하지 않는 경우

- 설명
  - 기준 브랜치를 혼자서 사용하는 경우
  - 또는 공동으로 사용하지만 작업 빈도가 적어서 내가 모르는 사이에 기준 브랜치에 새 커밋이 올라오지 않는 경우

- 작업
  1. 기준 브랜치에서 b 브랜치 생성
  2. b 브랜치에 c1 커밋 추가
  3. 기준 브랜치에서 b1 브랜치 머지


```bash
$ git init test_merge_1
Initialized empty Git repository in test_merge_1/.git/
$ cd test_merge_1 

$ echo "line 1" >> file1.txt
$ git add .
$ git commit -m c0
[main (root-commit) f36cdc2] c0
 1 file changed, 1 insertion(+)
 create mode 100644 file1.txt

$ git switch -c a
Switched to a new branch 'a'

$ echo "line 2" >> file1.txt
$ git add .
$ git commit -m c1
[a 68971af] c1
 1 file changed, 1 insertion(+)

$ git switch main
Switched to branch 'main'
$ git merge a -m "merge a"
Updating f36cdc2..68971af
Fast-forward (no commit created; -m option ignored)
 file1.txt | 1 +
 1 file changed, 1 insertion(+)
```

### 경우 3: 백머지를 하지 않아서 충돌이 발생하는 경우

- 작업 순서
  1. 기준 브랜치에서 b 브랜치 생성
  2. b 브랜치에 c1 커밋 추가 -> 다른 사람이 먼저 머지
  3. 기준 브랜치에 c2 커밋 추가
  4. 기준 브랜치에서 b1 브랜치 머지 -> 충돌
      - 만약 다른 사람이 기준 브랜치 머지를 하지 않으면 (2번이 없는 경우) 충돌이 발생하지 않음
  5. 기준 브랜치에서 머지 취소 (`merge --abort`)
  6. b 브랜치에서 기준 브랜치 머지 -> 충돌
  7. 충돌 해결
  8. 기준 브랜치에서 b1 브랜치 머지 -> 성공


### 경우 2: 백-머지로 충돌을 줄인 경우

- 설명
  - 레포를 공동으로 사용하는 경우
  - 기준 브랜치에 내가 모르는 사이에 커밋이 올라올 수 있다는 가정
  - 규칙: 작업 브랜치에 커밋하기 전에 항상 기준 브랜치를 백-머지(back-merge)해 주어야 함
  - 위 규칙을 지키면 충돌 가능성이 적어짐
  - 위 규칙을 지켜도 기준 브랜치에 머지 직전 다른 사람이 기준 브랜치에 커밋을 올릴 수도 있으므로 충돌 가능성은 존재
  - 공동 사용하는 사람이 적어서 기준 브랜치 머지 간격이 길면 충돌 가능성이 적어진다.

- 작업 순서
  1. 기준 브랜치에서 a 브랜치 생성
  2. 기준 브랜치에 c2 커밋 추가 -> 다른 사람이 먼저 머지
  3. a 브랜치에서 기준 브랜치 머지
  4. a 브랜치에 c1 커밋 추가
  5. 기준 브랜치에서 b1 브랜치 머지

### 경우 4: 백머지를 했지만 충돌이 발생하는 경우

- 설명
  - 레포를 공동으로 사용하는 경우
  - 작업 브랜치에 커밋하기 전에 백-머지(back-merge)를 했는데 그 이전에 다른 사람이 먼저 기준 브랜치에 커밋을 올린 경우
  - 백-머지시 충돌 발생
  - 충돌을 해결하고 커밋
  - 커밋 이후에 기준 브랜치에 머지
  - 이렇게 해도 백-머지와 기준 브랜치 머지 사이에 다른 사람이 먼저 먼저 기준 브랜치에 커밋을 올리면 충돌이 발생한다.
  - 많은 사람이 공동 사용해서 기준 브랜치 머지 간격이 아주 짧으면 충돌이 무한 반복될 수 있다.

- 작업 순서
  1. 기준 브랜치에서 b 브랜치 생성
  2. 기준 브랜치에 c2 커밋 추가 -> 다른 사람이 먼저 머지
  3. b 브랜치에 c1 커밋 추가
  4. b 브랜치에서 기준 브랜치 머지 -> 충돌
  5. 충돌 해결
  6. 기준 브랜치에서 b1 브랜치 머지
     - 만약 5번과 6번 사이에 다른 사람이 기준 브랜치 머지하면 또 충돌 발생

```bash
$ git init test_merge_4
Initialized empty Git repository in test_merge_4/.git/
$ cd test_merge_2

$ echo "line 1" >> file1.txt
$ git add .
$ git commit -m c0
[main (root-commit) f36cdc2] c0
 1 file changed, 1 insertion(+)
 create mode 100644 file1.txt

$ git switch -c a
Switched to a new branch 'a'

$ echo "line 2" >> file1.txt
$ git add .
$ git commit -m c1
[a fb6bd2c] c1
 1 file changed, 1 insertion(+)

$ git switch main
Switched to branch 'main'


$ echo "line 3" >> file1.txt
$ git add .
$ git commit -m c1
[main 7d13172] c2
 1 file changed, 1 insertion(+)

$ git merge a -m "merge a"
Auto-merging file1.txt
CONFLICT (content): Merge conflict in file1.txt
Automatic merge failed; fix conflicts and then commit the result.

$ git status
On branch main
You have unmerged paths.
  (fix conflicts and run "git commit")
  (use "git merge --abort" to abort the merge)

Unmerged paths:
  (use "git add <file>..." to mark resolution)
        both modified:   file1.txt

no changes added to commit (use "git add" and/or "git commit -a")
```

```txt
line 1
<<<<<<< HEAD
line 3
=======
line 2
>>>>>>> a
```